In [ ]:
import os
from itertools import product
import math

from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('max_colwidth', None)

In [ ]:
data_dir = '../data/'
output_dir = '../outputs/'
output_prefix = 'taka-toolo_'

## Read data

### Road names 

In [ ]:
for path, dirs, files in os.walk(data_dir):
    break

for i, fn in enumerate(files):
    print(i, fn)

In [ ]:
road_file = 'taka-toolo-roads.txt'

In [ ]:
with open(data_dir + road_file, 'r') as fo:
    roads_raw = fo.read().split('\n')
    
len(roads_raw), roads_raw[:4]

In [ ]:
def length_of_longest(roads_raw):
    return max([
        len(rn) for rn in roads_raw
    ])
longest_rn = length_of_longest(roads_raw)
longest_rn

### Finnish words corpus 

In [ ]:
dirs

In [ ]:
corpus_file = dirs[0] + '/kotus-sanalista_v1.xml'
corpus_file

In [ ]:
with open(data_dir + corpus_file, 'r') as fo:
    content = fo.read().split('\n')
    
    content = ''.join(content)
    bs_content = bs(content, 'lxml')

In [ ]:
corpus_raw = [tag.contents[0] for tag in bs_content.find_all('s')]
corpus_raw[:5]

In [ ]:
corpus_set = set(corpus_raw)

In [ ]:
len(corpus_raw)

## Building palindromes from the road names 

### Use letters several times 

In [ ]:
rn = roads_raw[0]
rn

In [ ]:
def toString(List):
    return ''.join(List)
 
# Function to print permutations of string
# This function takes three parameters:
# 1. String
# 2. Starting index of the string
# 3. Ending index of the string.
def permute(a, l, r, perms:list):
    if l==r:
        perms.append(toString(a))
    else:
        for i in range(l,r+1):
            a[l], a[i] = a[i], a[l]
            permute(a, l+1, r, perms)
            a[l], a[i] = a[i], a[l] # backtrack


In [ ]:
perm = []
permute(list('aur'), 0, len('aur')-1, perm)
perm = sorted(list(set(perm)))

In [ ]:
perm

In [ ]:
def all_permutations(string, up_to_n = 7):
    perms = []
    top = min(len(string), up_to_n)
    for i in range(1, top+1):
        print(f'permutations of length {i}/{len(string)}', end='\r')
        perms += [
            ''.join(comb) for comb in product(list(string), repeat=i)
        ]
        
    print('done!                                         ')

    return sorted(list(set(perms)))

In [ ]:
print(rn)
perms = all_permutations(rn.lower(), 7)

In [ ]:
len(perms)

In [ ]:
perms[:10]

#### Take only palindromes 

In [ ]:
def is_palindrome(s):
    is_pali = True

    l = len(s)
    for i in range(l // 2 + l % 2):
        #print(i, s[i], s[-i-1])
        if s[i] != s[-i-1]:
            is_pali=False
            break

    return is_pali

In [ ]:
def filter_palindromes(candidates):
    pali_mask = [
        is_palindrome(s) for s in candidates
    ]

    palinds = np.array(candidates)[pali_mask]
    return palinds

In [ ]:
palinds = filter_palindromes(perms)
palinds.shape

#### Check which ones include words 

In [ ]:
def filter_words(candidates, corpus_set):
    is_word_mask = [
        c in corpus_set for c in candidates
    ]

    words = np.array(candidates)[is_word_mask]
    return words

In [ ]:
filter_words(palinds, corpus_set)

### Palindromes for all roads in one area, letters can be reused 

In [ ]:
def get_tabs(rn:str, longest_rn:int, tab=6):
    t_str = ((longest_rn - len(rn)) // tab + 1) * ['\t']
    t_str = ''.join(t_str)
    return t_str

In [ ]:
def meaningful_palindromes(road_names, corpus_set, up_to_n=7, verbose = False):
    words = {}
    longest_rn = length_of_longest(road_names)
    
    for i, rn in enumerate(road_names):
        t_str = get_tabs(rn, longest_rn)
        if verbose: print(f'{rn}{t_str} {i+1}/{len(road_names)}')

        perms = all_permutations(rn.lower(), up_to_n)
        palinds = filter_palindromes(perms)
        words[rn] = filter_words(palinds, corpus_set)

    return words

In [ ]:
def get_unique_values(words):
    return sorted(list(set(np.concatenate(list(words.values())).tolist())))

In [ ]:
up_to_n = 6

In [ ]:
%%time
words = meaningful_palindromes(roads_raw, 
                               corpus_set,
                               up_to_n=up_to_n, 
                               verbose=True)
words_unique = get_unique_values(words)

In [ ]:
print(f'for {len(words)} road names we found {len(words_unique)} unique, meaningful palindroms with max length of {up_to_n}')

In [ ]:
words_unique

In [ ]:
road2n = {
    key: len(words[key]) for key in words
}

road2n

In [ ]:
road2words = {
    key: ', '.join(words[key]) for key in words
}

road2words

#### Results to DF 

In [ ]:
df_words = pd.DataFrame(road2n, index=['n_words']).T.reset_index().rename(columns={'index': 'street'})
df_words['words'] = list(road2words.values())

df_words.head()

In [ ]:
df_words.sort_values('n_words', ascending=False)

#### Save results

In [ ]:
df_words.to_csv(output_dir + output_prefix + 'palindrome_words_by_street.csv')
with open(output_dir + output_prefix + 'palindrome_words_unique.txt', 'w') as fo:
    fo.write('\n'.join(words_unique))

### Meaningful words from letters available in one road name

In [ ]:
def meaningful_words(road_names, corpus_set, up_to_n=7, verbose=True):
    words = {}
    longest_rn = length_of_longest(road_names)
    
    for i, rn in enumerate(road_names):
        t_str = get_tabs(rn, longest_rn)
        if verbose: print(f'{rn}{t_str} {i+1}/{len(road_names)}')

        perms = all_permutations(rn.lower(), up_to_n)
        words[rn] = filter_words(perms, corpus_set)

    return words

In [ ]:
%%time
words_all = meaningful_words(roads_raw, corpus_set,
                        up_to_n=up_to_n, 
                        verbose=True)

words_all_unique = get_unique_values(words_all)

In [ ]:
print(f'for {len(roads_raw)} road names we found {len(words_all_unique)} unique words with max length of {up_to_n}')

### Use letter only once 

In [ ]:
def palindromeSubStrs(s):
    m = dict()
    n = len(s)
  
    # table for storing results (2 rows for odd-
    # and even-length palindromes
    R = [[0 for x in range(n+1)] for x in range(2)]
  
    # Find all sub-string palindromes from the given input
    # string insert 'guards' to iterate easily over s
    s = "@" + s + "#"

    for j in range(2):
        rp = 0    # length of 'palindrome radius'
        R[j][0] = 0
  
        i = 1
        while i <= n:
  
            # Attempt to expand palindrome centered at i
            while s[i - rp - 1] == s[i + j + rp]:
                rp += 1 # Incrementing the length of palindromic
                        # radius as and when we find valid palindrome
  
            # Assigning the found palindromic length to odd/even
            # length array
            R[j][i] = rp
            k = 1
            while (R[j][i - k] != rp - k) and (k < rp):
                R[j][i+k] = min(R[j][i-k], rp - k)
                k += 1
            rp = max(rp - k, 0)
            i += k
  
    # remove guards
    s = s[1:len(s)-1]
  
    # Put all obtained palindromes in a hash map to
    # find only distinct palindrome
    m[s[0]] = 1
    for i in range(1,n):
        for j in range(2):
            for rp in range(R[j][i],0,-1):
                m[s[i - rp - 1 : i - rp - 1 + 2 * rp + j]] = 1
        m[s[i]] = 1
  
    
    return sorted(list(m.keys()))

In [ ]:
def palinds_use_letters_once(road_names):
    palinds = {}
    
    for rn in road_names:
        palinds[rn] = palindromeSubStrs(rn.lower())

    return palinds

In [ ]:
palinds = palinds_use_letters_once(roads_raw)

palinds_unique = get_unique_values(palinds)
palinds_unique

In [ ]:
filter_words(palinds_unique, corpus_set)